## Load Workload stats XLS and convert to RAC logbook format

#### Converting workload stats into RAC format is a real pain....

<img src='images/workload_excel.jpg' width="800" height="400">

<img src='images/workload_rac.jpg' width="800" height="400">

## This is where we can use Python!

<img src='images/automate_boring_stuff.jpg' width=500>

#### Let's get started by loading the data from the Excel file using the Pandas library

We'll open `sample.xlsx`, and display the contents

In [5]:
import pandas as pd

xls_file = 'sample.xlsx'
df = pd.read_excel(xls_file, header=2)

df = df.dropna()

print(df)

                Modality                                     Procedure Name  \
1    Computed Tomography                      CT Aortogram (Chest, Abdomen)   
2    Computed Tomography                                         CT Abdomen   
3    Computed Tomography                                         CT Kidneys   
4    Computed Tomography                                           CT Liver   
5    Computed Tomography                                        CT Pancreas   
..                   ...                                                ...   
168    Radio Fluoroscopy                      Operating Theatre Fluoroscopy   
169    Radio Fluoroscopy              Fluoroscopy for Transbronchial Biopsy   
170    Radio Fluoroscopy                    Video Fluoroscopy Swallow Study   
171    Radio Fluoroscopy  Water-soluble Contrast follow through for adhe...   
173           Ultrasound                                  US Pelvis, Female   

    RVU Per Procedure  Paediatric cases  Adult Case

##### This quite busy.  Let's simplify by only keeping the columns we care about

##### and only keeping the CT cases



In [6]:
df = df[['Modality', 'Procedure Name', 'Total Read']]

df_ct = df[df['Modality'] == 'Computed Tomography']

print(df_ct)


               Modality                    Procedure Name  Total Read
1   Computed Tomography     CT Aortogram (Chest, Abdomen)         2.0
2   Computed Tomography                        CT Abdomen         4.0
3   Computed Tomography                        CT Kidneys         3.0
4   Computed Tomography                          CT Liver         2.0
5   Computed Tomography                       CT Pancreas         1.0
6   Computed Tomography             CT Abdomen and Pelvis        56.0
7   Computed Tomography           CT Aortogram (Thoracic)         2.0
8   Computed Tomography             CT Coronary Angiogram         1.0
9   Computed Tomography           CT Mesenteric Angiogram         3.0
10  Computed Tomography                          CT Brain       119.0
11  Computed Tomography                          CT Chest        18.0
12  Computed Tomography              CT Chest and Abdomen         1.0
13  Computed Tomography      CT Chest, Abdomen and Pelvis        22.0
14  Computed Tomogra

### Now it's much easier to see what is going on.

##### Next: How to group which procedures into which RAC bucket? <img src='images/round_peg.jpg' width=300>

We need to match the procedure names into one (or more) of the RAC buckets.

--disclaimer-- I made this bit up.  Do not use this method for your own data...

In [8]:
lookup_dict = {
    'CT Aortogram (Chest, Abdomen)': ['Aortogram'],
    'CT Abdomen': ['Abodmen (GI)'],
    'CT Kidneys': ['Urinary'],
    'CT Liver': ['Abodmen (GI)'],
    'CT Pancreas': ['Abodmen (GI)'],
    'CT Abdomen and Pelvis': ['Abodmen (GI)', 'Pelvis'],
    'CT Aortogram (Thoracic)': ['Aortogram'],
    'CT Coronary Angiogram': ['Coronary'],
    'CT Mesenteric Angiogram': ['Aortogram'],
    'CT Brain': ['Brain'],
    'CT Chest': ['Thorax'],
    'CT Chest and Abdomen': ['Thorax','Abdomen (GI)'],
    'CT Chest, Abdomen and Pelvis': ['Thorax', 'Abdomen (GI)', 'Pelvis'],
    'CT Chest, High Resolution': ['HRCT'],
    'CT Chest (Pulmonary Angiography)': ['Pulm Angio'],
    'CT Chest Wall': ['Thorax'],
    'CT Face (Facial Bone)': ['Head & Neck'],
    'CT Right Foot': ['Limbs'],
    'CT Urography': ['Urinary'],
    'CT Left Knee': ['Limbs'],
    'CT Temporal Bone': ['Head & Neck'],
    'CT Neck': ['Head & Neck'],
    'CT Colonography': ['CT Colono'],
    'CT Pelvis': ['Pelvis'],
    'CT Pelvis Bony': ['Pelvis'],
    'CT Salivary Glands, Parotid': ['Head & Neck'],
    'CT Paranasal Sinuses': ['Head & Neck'],
    'CT Cervical Spine': ['Spine'],
    'CT KUB (Non Contrast)': ['Urinary'],
    'CT Right Wrist': ['Limbs'],
}

In [9]:
lookup_dict['CT Right Wrist']

['Limbs']

In [10]:
lookup_dict['CT Chest, Abdomen and Pelvis']

['Thorax', 'Abdomen (GI)', 'Pelvis']

In [11]:
for item in lookup_dict['CT Chest, Abdomen and Pelvis']:
    print(f'\t{item}')

	Thorax
	Abdomen (GI)
	Pelvis


#### So we can associate the Procedure Name with the destination(s) within `lookup_dict`

<img src='images/lookup_diag.jpg' width=800>

#### Now we set up a different kind of dictionary where we can store our output data

This is a convenience which reduces complexity later on.

In [12]:
from collections import defaultdict

def default_value():
    return 0

rac = defaultdict(default_value)


In [13]:
print( rac.keys() )
print( rac['Brain'] )
print( rac.keys() )

dict_keys([])
0
dict_keys(['Brain'])


#### This is actual work of the algorithm here:

We iterate through the list of studies reported, and show where the number of studies will be added to.

In [14]:
# loop through input workload and add to the dict

ct_only = df[df['Modality']=='Computed Tomography']

for idx, row in ct_only.iterrows():
    p_name = lookup_dict[row['Procedure Name']]
    print(f"{idx}. {row['Procedure Name']:<30}\t\t+ {row['Total Read']:.0f} to: {p_name}")

1. CT Aortogram (Chest, Abdomen) 		+ 2 to: ['Aortogram']
2. CT Abdomen                    		+ 4 to: ['Abodmen (GI)']
3. CT Kidneys                    		+ 3 to: ['Urinary']
4. CT Liver                      		+ 2 to: ['Abodmen (GI)']
5. CT Pancreas                   		+ 1 to: ['Abodmen (GI)']
6. CT Abdomen and Pelvis         		+ 56 to: ['Abodmen (GI)', 'Pelvis']
7. CT Aortogram (Thoracic)       		+ 2 to: ['Aortogram']
8. CT Coronary Angiogram         		+ 1 to: ['Coronary']
9. CT Mesenteric Angiogram       		+ 3 to: ['Aortogram']
10. CT Brain                      		+ 119 to: ['Brain']
11. CT Chest                      		+ 18 to: ['Thorax']
12. CT Chest and Abdomen          		+ 1 to: ['Thorax', 'Abdomen (GI)']
13. CT Chest, Abdomen and Pelvis  		+ 22 to: ['Thorax', 'Abdomen (GI)', 'Pelvis']
14. CT Chest, High Resolution     		+ 2 to: ['HRCT']
15. CT Chest (Pulmonary Angiography)		+ 6 to: ['Pulm Angio']
16. CT Chest Wall                 		+ 1 to: ['Thorax']
17. CT Face (Facial Bone)        

<img src='images/lookup_diag2.jpg' width=800>

In [15]:
# loop through input workload and add to the dict

ct_only = df[df['Modality']=='Computed Tomography']

for idx, row in ct_only.iterrows():
    p_name = lookup_dict[row['Procedure Name']]
    print(f"{idx}. {row['Procedure Name']:<30}")#   \t\tAdd to column: {p_name}")

    for rac_bucket in p_name:
        print(f'{40*" "}{rac_bucket}\t+ {row["Total Read"]:.0f}')


1. CT Aortogram (Chest, Abdomen) 
                                        Aortogram	+ 2
2. CT Abdomen                    
                                        Abodmen (GI)	+ 4
3. CT Kidneys                    
                                        Urinary	+ 3
4. CT Liver                      
                                        Abodmen (GI)	+ 2
5. CT Pancreas                   
                                        Abodmen (GI)	+ 1
6. CT Abdomen and Pelvis         
                                        Abodmen (GI)	+ 56
                                        Pelvis	+ 56
7. CT Aortogram (Thoracic)       
                                        Aortogram	+ 2
8. CT Coronary Angiogram         
                                        Coronary	+ 1
9. CT Mesenteric Angiogram       
                                        Aortogram	+ 3
10. CT Brain                      
                                        Brain	+ 119
11. CT Chest                      
                        

In [16]:
# Re-define this - reset our  back to zero
rac = defaultdict(default_value)

# loop through input workload and add to the dict

ct_only = df[df['Modality']=='Computed Tomography']

for idx, row in ct_only.iterrows():
    p_name = lookup_dict[row['Procedure Name']]

    for col in p_name:
        rac[col] = int(rac[col] + row['Total Read'])

rac

defaultdict(<function __main__.default_value()>,
            {'Aortogram': 7,
             'Abodmen (GI)': 63,
             'Urinary': 17,
             'Pelvis': 80,
             'Coronary': 1,
             'Brain': 119,
             'Thorax': 42,
             'Abdomen (GI)': 23,
             'HRCT': 2,
             'Pulm Angio': 6,
             'Head & Neck': 39,
             'Limbs': 3,
             'CT Colono': 1,
             'Spine': 2})

In [17]:
rac_df = pd.DataFrame(rac, index=[0])
rac_df

,Aortogram,Abodmen (GI),Urinary,Pelvis,Coronary,Brain,Thorax,Abdomen (GI),HRCT,Pulm Angio,Head & Neck,Limbs,CT Colono,Spine
0,7,63,17,80,1,119,42,23,2,6,39,3,1,2


# Great!  But what which quarter of which year is this data from?

### We can't access this info from the data we have loaded above, so we need to access the XLS in a slightly

### different way - this allows us to see the 'raw' data in the excel sheet.

<img src='images/workload_excel.jpg' width="800" height="400">


In [18]:
import openpyxl

wb_obj = openpyxl.load_workbook(xls_file)
 
# Get workbook active sheet object from the active attribute
sheet_obj = wb_obj.active

sheet_obj['B1'].value

'Approved Date Period:01/01/2022 to 31/03/2022'

In [19]:
print(sheet_obj['A4'].value)

Dr Dave (12345A)


### OK: Cell B1 holds the dates, and A4 holds the resident name & MCR

##### Getting the name will be easy - we're not going to bother separating MCR out

In [20]:
rac['Name'] = sheet_obj['A4'].value


### Extracting and interpreting the dates in B1 is a bit harder, but not much!

##### The library `datefinder` makes this hugely easy.  We just feed the string with both dates in, and we get the computer friendly version of BOTH dates coming out!

In [23]:
import datefinder
import datetime as dt

# input_string = "I starting working on July 3rd 1997 8am, after which I gave up on 09 Dec 2005 3:45pm.  Working was no fun."
input_string = sheet_obj['B1'].value

date_list = list(datefinder.find_dates(input_string))
print(f'Input:  {input_string}')
print(f'Output: {date_list}')

Input:  Approved Date Period:01/01/2022 to 31/03/2022
Output: [datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2022, 3, 31, 0, 0)]


#### Before we set the results in stone, we should check our dates first.  Do we have data from dates that will fit into the RAC expectations?  If not we will have to re-extract.  That is a pain.

#### Let's check, and turn it into a nice way to generate the 'Date' column text.

#### We want to end up with 'YYYY Qn' e.g. '2021 Q3'

In [25]:
# These are the 2 dates from above
a = date_list[0]
b = date_list[1]
print(f'a:{a:%d-%m-%Y}, b:{b:%d-%m-%Y} (DD-MM-YYYY)')

# This function will check the years match, and that both dates are within the same quarter
# Then will return the 'YYYY Qn' that we want
def get_q(a, b):
    if a.year != b.year:
        print('[get_q] Different Years')
        return f'{a:%d-%m-%Y} --> {b:%d-%m-%Y}'
    else:
        if a.month in [1,2,3] and b.month in [1,2,3]:
            q = 'Q1'
        elif a.month in [4,5,6] and b.month in [4,5,6]:
            q = 'Q2'
        elif a.month in [7,8,9] and b.month in [7,8,9]:
            q = 'Q3'
        elif a.month in [10,11,12] and b.month in [10,11,12]:
            q = 'Q4'
        else:
            print('[get_q] Different quarters')
            return f'{a:%d-%m-%Y} --> {b:%d-%m-%Y}'
        return f'{a.year} {q}'

    
get_q(date_list[0], date_list[1])

a:01-01-2022, b:31-03-2022 (DD-MM-YYYY)


'2022 Q1'

### Nice. Our data is all from a single quarter.

#### What happens if it isn't?

Let's see what happens if we use `a` and `c` which are _not_ in the same quarter...

In [26]:
c = date_list[0]
c = dt.date(day=15, month=7, year=2021)

print(f'{c:%d-%m-%Y} (DD-MM-YYYY)')

print(get_q(a, c))

15-07-2021 (DD-MM-YYYY)
[get_q] Different Years
01-01-2022 --> 15-07-2021


### We take `a` and `c` as the date range start & end.

### This kicks up a warning and spits out the data with the specific date range rather than a single quarter.

The user may need to get the data re-extracted to fit the date ranges the RAC needs.

### Let's complete the data and show the final table output

In [27]:
rac['Date'] = get_q(a, b)

rac_df = pd.DataFrame(rac, index=[0])

rac_df

,Aortogram,Abodmen (GI),Urinary,Pelvis,Coronary,Brain,Thorax,Abdomen (GI),HRCT,Pulm Angio,Head & Neck,Limbs,CT Colono,Spine,Name,Date
0,7,63,17,80,1,119,42,23,2,6,39,3,1,2,Dr Dave (12345A),2022 Q1


### But those columns are not in the same order!?! That's going to be horrible to transcribe!


In [28]:
col_order = [
 'Name',
 'Date',
 'Brain',
 'Head & Neck',
 'Thorax',
 'HRCT',
 'Pulm Angio',
 'Aortogram',
 'Coronary',
 'Abodmen (GI)',
 'CT Colono',
 'Urinary',
 'Pelvis',
 'Spine',
 'Limbs',
]

rac_df = rac_df[col_order]

rac_df

,Name,Date,Brain,Head & Neck,Thorax,HRCT,Pulm Angio,Aortogram,Coronary,Abodmen (GI),CT Colono,Urinary,Pelvis,Spine,Limbs
0,Dr Dave (12345A),2022 Q1,119,39,42,2,6,7,1,63,1,17,80,2,3


### That's what we want!


### But we have been beaten to it!  Dr Tranh Anh did this before! (He used R, not Python)

<img src='images/AnhRACLogbookWebsite.jpg' width=800>

### All that code (a bit tidied up) is below.

In [29]:
import pandas as pd
import openpyxl
import datefinder
import datetime as dt
from collections import defaultdict

xls_file = 'sample.xlsx'

df = pd.read_excel(xls_file, header=2)

# Initial tidy up
df = df.dropna()
df = df[['Modality', 'Procedure Name', 'Total Read']]

# Let's only look at CT
ct_only = df[df['Modality'] == 'Computed Tomography']

lookup_dict = {
    'CT Aortogram (Chest, Abdomen)': ['Aortogram'],
    'CT Abdomen': ['Abodmen (GI)'],
    'CT Kidneys': ['Urinary'],
    'CT Liver': ['Abodmen (GI)'],
    'CT Pancreas': ['Abodmen (GI)'],
    'CT Abdomen and Pelvis': ['Abodmen (GI)', 'Pelvis'],
    'CT Aortogram (Thoracic)': ['Aortogram'],
    'CT Coronary Angiogram': ['Coronary'],
    'CT Mesenteric Angiogram': ['Aortogram'],
    'CT Brain': ['Brain'],
    'CT Chest': ['Thorax'],
    'CT Chest and Abdomen': ['Thorax','Abdomen (GI)'],
    'CT Chest, Abdomen and Pelvis': ['Thorax', 'Abdomen (GI)', 'Pelvis'],
    'CT Chest, High Resolution': ['HRCT'],
    'CT Chest (Pulmonary Angiography)': ['Pulm Angio'],
    'CT Chest Wall': ['Thorax'],
    'CT Face (Facial Bone)': ['Head & Neck'],
    'CT Right Foot': ['Limbs'],
    'CT Urography': ['Urinary'],
    'CT Left Knee': ['Limbs'],
    'CT Temporal Bone': ['Head & Neck'],
    'CT Neck': ['Head & Neck'],
    'CT Colonography': ['CT Colono'],
    'CT Pelvis': ['Pelvis'],
    'CT Pelvis Bony': ['Pelvis'],
    'CT Salivary Glands, Parotid': ['Head & Neck'],
    'CT Paranasal Sinuses': ['Head & Neck'],
    'CT Cervical Spine': ['Spine'],
    'CT KUB (Non Contrast)': ['Urinary'],
    'CT Right Wrist': ['Limbs'],
}


# Make our initial dict to store data before adding to the DF
def default_value():
    return 0

rac = defaultdict(default_value)

# Run the numbers
for idx, row in ct_only.iterrows():
    p_name = lookup_dict[row['Procedure Name']]

    for col in p_name:
        rac[col] = int(rac[col] + row['Total Read'])


# Get Name and Dates
wb_obj = openpyxl.load_workbook(xls_file)
sheet_obj = wb_obj.active

date_list = list(datefinder.find_dates(sheet_obj['B1'].value))

def get_q(a, b):
    if a.year != b.year:
        print('[get_q] Different Years')
        return f'{a:%d-%m-%Y} --> {b:%d-%m-%Y}'
    else:
        if a.month in [1,2,3] and b.month in [1,2,3]:
            q = 'Q1'
        elif a.month in [4,5,6] and b.month in [4,5,6]:
            q = 'Q2'
        elif a.month in [7,8,9] and b.month in [7,8,9]:
            q = 'Q3'
        elif a.month in [10,11,12] and b.month in [10,11,12]:
            q = 'Q4'
        else:
            print('[get_q] Different quarters')
            return f'{a:%d-%m-%Y} --> {b:%d-%m-%Y}'
        return f'{a.year} {q}'


rac['Name'] = sheet_obj['A4'].value
rac['Date'] = get_q(date_list[0], date_list[1])

rac_df = pd.DataFrame(rac, index=[0])

col_order = [
 'Name',
 'Date',
 'Brain',
 'Head & Neck',
 'Thorax',
 'HRCT',
 'Pulm Angio',
 'Aortogram',
 'Coronary',
 'Abodmen (GI)',
 'CT Colono',
 'Urinary',
 'Pelvis',
 'Spine',
 'Limbs',
]

# Final Result!
rac_df = rac_df[col_order]

rac_df

,Name,Date,Brain,Head & Neck,Thorax,HRCT,Pulm Angio,Aortogram,Coronary,Abodmen (GI),CT Colono,Urinary,Pelvis,Spine,Limbs
0,Dr Dave (12345A),2022 Q1,119,39,42,2,6,7,1,63,1,17,80,2,3


## And Finally...

#### Here's a low-rent <u><b>online</b></u> version using exactly this code!

### <a href="https://r4lectures.pythonanywhere.com">r4lectures.pythonanywhere.com</a>

<img src='images/qrcode.jpeg' width=300>
